In [2]:
import pymongo
import json
from datetime import datetime, date, time

In [3]:
mng_client = pymongo.MongoClient('localhost', 27017)
mng_db = mng_client['local']
db_cm = mng_db['my_collection']

получение списка поездок, которые произошли в указанном промежутке

In [4]:
def get_drives_by_time(start_time, finish_time):
    ret_list = []
    for item in db_cm.find({"tpep_pickup_datetime": {"$gt": start_time, "$lt": finish_time}}):
        ret_list.append(item) 
    return ret_list

In [5]:
%%time
get_drives_by_time(1514766560000, 1514766569999)

Wall time: 6.65 s


[{'_id': ObjectId('5beaf7a1afb13306c092d923'),
  'VendorID': 1,
  'tpep_pickup_datetime': 1514766569000,
  'tpep_dropoff_datetime': 1514766768000,
  'passenger_count': 3,
  'trip_distance': 0.5,
  'RatecodeID': 1,
  'store_and_fwd_flag': False,
  'PULocationID': 143,
  'DOLocationID': 143,
  'payment_type': 2,
  'fare_amount': 4.5,
  'extra': 0.5,
  'mta_tax': 0.5,
  'tip_amount': 0.0,
  'tolls_amount': 0.0,
  'improvement_surcharge': 0.3,
  'total_amount': 5.8},
 {'_id': ObjectId('5beaf7a2afb13306c092d97b'),
  'VendorID': 2,
  'tpep_pickup_datetime': 1514766564000,
  'tpep_dropoff_datetime': 1514767735000,
  'passenger_count': 1,
  'trip_distance': 4.2,
  'RatecodeID': 1,
  'store_and_fwd_flag': False,
  'PULocationID': 65,
  'DOLocationID': 186,
  'payment_type': 2,
  'fare_amount': 16.5,
  'extra': 0.5,
  'mta_tax': 0.5,
  'tip_amount': 0.0,
  'tolls_amount': 0.0,
  'improvement_surcharge': 0.3,
  'total_amount': 17.8},
 {'_id': ObjectId('5beaf7a2afb13306c092d9e3'),
  'VendorID': 2,

постраничный вывод данных о поездках

In [4]:
def pagination(page_size, pages, last_id=None, cur_page=0):
    if cur_page <= pages:
        if last_id is None:
            cursor = db_cm.find().limit(page_size)
        else:
            cursor = db_cm.find({'_id': {'$gt': last_id}}).limit(page_size)

        data = [x for x in cursor]

        if not data:
            return None, None

        last_id = data[-1]['_id']
        return pagination(page_size, pages, last_id, cur_page+1)

In [5]:
%%time
pagination(50, 3)

Wall time: 6.01 ms


вывести список зон с количеством начал и окончаний поездок в этой зоне

In [6]:
from bson.son import SON
pipeline = [
    {"$group": {"PULocationID": "$tags", "count": {"$sum": 1}}},
    {"$sort": SON([("count", -1), ("PULocationID", -1)])}
]
import pprint
pprint.pprint(list(db_cm.things.aggregate(pipeline)))

OperationFailure: The field 'PULocationID' must be an accumulator object